In [177]:
# Data Processing
import re
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

# NLP
from wordcloud import WordCloud

import spacy
nlp = spacy.load("es_core_news_sm")

# Others
import warnings
warnings.filterwarnings(action='ignore')

## Dataset

In [178]:
df = pd.read_csv('../data/raw/google_maps_all_reviews.csv')

In [179]:
print("Shape:", df.shape)

Shape: (3476, 8)


In [180]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3476 entries, 0 to 3475
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   place_id   3476 non-null   object 
 1   place_url  3476 non-null   object 
 2   review_id  3476 non-null   object 
 3   user_url   3476 non-null   object 
 4   username   0 non-null      float64
 5   stars      3476 non-null   int64  
 6   time       3474 non-null   object 
 7   text       3476 non-null   object 
dtypes: float64(1), int64(1), object(6)
memory usage: 217.4+ KB


In [181]:
df.drop(columns=["place_url", "user_url", "username", "review_id"], inplace=True)

In [182]:
df

,place_id,stars,time,text
0,club_majadas,5,Hace 3 meses,Costo beneficio correcto. Limpio y comodo de u...
1,club_majadas,5,Hace 5 años,Buena atención del personal. Muy higiénico. Ri...
2,club_majadas,5,Hace un año,Me encanto el lugar super recomendado La amabi...
3,club_majadas,5,Hace 8 años,"Un lugar para ejercitarse en el gimnasio, clas..."
4,club_majadas,5,Hace 3 años,"Un bello lugar para respirar aire puro, nadar,..."
...,...,...,...,...
3471,club_guatemala,5,Hace 7 años,Jose Miguel Montes Local Guide · 26 opiniones ...
3472,club_guatemala,3,Hace 8 años,Sandra Carmina Ventura Roldan       Hace...
3473,club_guatemala,4,Hace 8 años,Giovanni Ramirez Garcia Local Guide · 78 opini...
3474,club_guatemala,4,Hace 8 años,Sarita Arana       Hace 8 años


## Filtrado

In [183]:
def normalize_text(text):
    if not isinstance(text, str):
        try:
            import pandas as _pd
            if _pd.isna(text):
                text = ''
            else:
                text = str(text)
        except Exception:
            text = '' if text is None else str(text)

    text = text.lower()                         # Convert to lowercase
    text = re.sub(r'http\S+', '', text)         # Remove URLs
    text = re.sub(r'@\w+', '', text)            # Remove mentions
    text = re.sub(r'#\w+', '', text)            # Remove hashtags
    text = re.sub(r'\s+', ' ', text).strip()    # Remove extra whitespace
    
    convert_dict = {'á': 'a', 'é': 'e', 'í': 'i', 'ó': 'o', 'ú': 'u', 'ü': 'u', 'ñ': 'ni'}
    for accented_char, normal_char in convert_dict.items():
        text = text.replace(accented_char, normal_char)

    return text

In [184]:
df["clean_text"] = df["text"].apply(normalize_text)

In [185]:
check_1 = pd.DataFrame()

check_1 = df[df['clean_text'].str.contains('·')]

print("Removing rows with '·' in text:", check_1.shape[0])
df = df.drop(check_1.index)
print("New shape:", df.shape)

Removing rows with '·' in text: 732
New shape: (2744, 5)


In [186]:
check_2 = pd.DataFrame()

re_expression = [r'hace \d+ anios', r'hace un anio', r'hace un mes', r'hace \d+ meses']

for expr in re_expression:
    temp = df[df['clean_text'].str.contains(expr)]
    check_2 = pd.concat([check_2, temp])
    print(f"Number of rows with '{expr}':", temp.shape[0])

print("\nRemoving", check_2.shape[0], "rows from df")
df = df.drop(index=check_2.index)
print("New shape:", df.shape)

Number of rows with 'hace \d+ anios': 515
Number of rows with 'hace un anio': 32
Number of rows with 'hace un mes': 2
Number of rows with 'hace \d+ meses': 13

Removing 562 rows from df
New shape: (2182, 5)


In [187]:
df

,place_id,stars,time,text,clean_text
0,club_majadas,5,Hace 3 meses,Costo beneficio correcto. Limpio y comodo de u...,costo beneficio correcto. limpio y comodo de u...
1,club_majadas,5,Hace 5 años,Buena atención del personal. Muy higiénico. Ri...,buena atencion del personal. muy higienico. ri...
2,club_majadas,5,Hace un año,Me encanto el lugar super recomendado La amabi...,me encanto el lugar super recomendado la amabi...
3,club_majadas,5,Hace 8 años,"Un lugar para ejercitarse en el gimnasio, clas...","un lugar para ejercitarse en el gimnasio, clas..."
4,club_majadas,5,Hace 3 años,"Un bello lugar para respirar aire puro, nadar,...","un bello lugar para respirar aire puro, nadar,..."
...,...,...,...,...,...
3299,club_guatemala,5,Hace un año,Excelente,excelente
3303,club_guatemala,5,Hace 2 años,bantab,bantab
3305,club_guatemala,5,Hace 5 años,Desde,desde
3306,club_guatemala,5,Hace 7 años,Cicatrización,cicatrizacion


## Pre Procesamiento

### Valor "text"

In [188]:
def split_into_sentences(text):
    sentences = re.split(r'[.!?]+', text)
    
    cleaned_sentences = []
    for sentence in sentences:
        sentence = sentence.strip()
        sentence = re.sub(r'[^a-zA-Z0-9áéíóúüñÁÉÍÓÚÜÑ\s]', '', sentence)
        if sentence and sentence != '':
            cleaned_sentences.append(sentence)
    
    return cleaned_sentences

In [189]:
df['sentences'] = df['clean_text'].apply(split_into_sentences)

In [190]:
df

,place_id,stars,time,text,clean_text,sentences
0,club_majadas,5,Hace 3 meses,Costo beneficio correcto. Limpio y comodo de u...,costo beneficio correcto. limpio y comodo de u...,"[costo beneficio correcto, limpio y comodo de ..."
1,club_majadas,5,Hace 5 años,Buena atención del personal. Muy higiénico. Ri...,buena atencion del personal. muy higienico. ri...,"[buena atencion del personal, muy higienico, r..."
2,club_majadas,5,Hace un año,Me encanto el lugar super recomendado La amabi...,me encanto el lugar super recomendado la amabi...,[me encanto el lugar super recomendado la amab...
3,club_majadas,5,Hace 8 años,"Un lugar para ejercitarse en el gimnasio, clas...","un lugar para ejercitarse en el gimnasio, clas...",[un lugar para ejercitarse en el gimnasio clas...
4,club_majadas,5,Hace 3 años,"Un bello lugar para respirar aire puro, nadar,...","un bello lugar para respirar aire puro, nadar,...",[un bello lugar para respirar aire puro nadar ...
...,...,...,...,...,...,...
3299,club_guatemala,5,Hace un año,Excelente,excelente,[excelente]
3303,club_guatemala,5,Hace 2 años,bantab,bantab,[bantab]
3305,club_guatemala,5,Hace 5 años,Desde,desde,[desde]
3306,club_guatemala,5,Hace 7 años,Cicatrización,cicatrizacion,[cicatrizacion]


In [191]:
def lemmatize_spacy(sentences):
    if isinstance(sentences, str):
        sentences = [sentences]
    if not isinstance(sentences, (list, tuple)):
        return []

    out = []
    for s in sentences:
        doc = nlp(s)
        lem = " ".join([token.lemma_ for token in doc if token.is_alpha])
        out.append(lem)
    return out

In [192]:
df['lemm_sentences'] = df['sentences'].apply(lemmatize_spacy)

In [193]:
nan_rows = df[df['lemm_sentences'].isna()]
nan_rows

,place_id,stars,time,text,clean_text,sentences,lemm_sentences


In [194]:
df = df.dropna(subset=['lemm_sentences'])
print("New shape:", df.shape)

New shape: (2182, 7)


### Valor "time"

In [195]:
current_year = datetime.datetime.now().year

In [196]:
def extract_year(time_str):
    if isinstance(time_str, str):
        time_str = time_str.lower().strip()
        time_str = time_str.replace("editado", "").strip()
        if "hace" in time_str:
            parts = time_str.split()
            if len(parts) == 3:
                try:
                    number = int(parts[1]) if parts[1].isdigit() else 1
                    unit = parts[2]
                    if "anio" in unit:
                        return current_year - number
                    elif "mes" in unit:
                        months_ago = number
                        year_adjustment = months_ago // 12
                        return current_year - year_adjustment
                    elif "semanas" in unit or "semana" in unit:
                        return current_year
                    elif "dias" in unit or "dia" in unit:
                        return current_year
                except ValueError:
                    return None
    return None

In [197]:
df["time"] = df["time"].apply(normalize_text)

In [198]:
df['real_time'] = df['time'].apply(extract_year)

In [199]:
nan_rows = df[df['real_time'].isna()]
nan_rows

,place_id,stars,time,text,clean_text,sentences,lemm_sentences,real_time
277,club_espaniol,4,,Excelente lugar amplio parqueo,excelente lugar amplio parqueo,[excelente lugar amplio parqueo],[excelente lugar amplio parqueo],NaN
319,club_espaniol,5,2 semanas atras,Excelente lugar,excelente lugar,[excelente lugar],[excelente lugar],NaN
1155,club_montania,5,,Un lugar muy bonito.,un lugar muy bonito.,[un lugar muy bonito],[uno lugar mucho bonito],NaN


In [200]:
df = df.dropna(subset=['real_time'])
print("New shape:", df.shape)

New shape: (2179, 8)


### Valor "stars"

In [201]:
df['real_stars'] = df['stars'] / 5.0

In [202]:
nan_rows = df[df['real_time'].isna()]
nan_rows

,place_id,stars,time,text,clean_text,sentences,lemm_sentences,real_time,real_stars


In [203]:
df = df.dropna(subset=['real_stars'])
print("New shape:", df.shape)

New shape: (2179, 9)


In [204]:
df

,place_id,stars,time,text,clean_text,sentences,lemm_sentences,real_time,real_stars
0,club_majadas,5,hace 3 meses,Costo beneficio correcto. Limpio y comodo de u...,costo beneficio correcto. limpio y comodo de u...,"[costo beneficio correcto, limpio y comodo de ...","[costo beneficio correcto, limpio y comodo de ...",2025.0,1.0
1,club_majadas,5,hace 5 anios,Buena atención del personal. Muy higiénico. Ri...,buena atencion del personal. muy higienico. ri...,"[buena atencion del personal, muy higienico, r...","[buen atencion del personal, mucho higienico, ...",2020.0,1.0
2,club_majadas,5,hace un anio,Me encanto el lugar super recomendado La amabi...,me encanto el lugar super recomendado la amabi...,[me encanto el lugar super recomendado la amab...,[yo encantar el lugar super recomendado el ama...,2024.0,1.0
3,club_majadas,5,hace 8 anios,"Un lugar para ejercitarse en el gimnasio, clas...","un lugar para ejercitarse en el gimnasio, clas...",[un lugar para ejercitarse en el gimnasio clas...,[uno lugar para ejercitar él en el gimnasio cl...,2017.0,1.0
4,club_majadas,5,hace 3 anios,"Un bello lugar para respirar aire puro, nadar,...","un bello lugar para respirar aire puro, nadar,...",[un bello lugar para respirar aire puro nadar ...,[uno bello lugar para respirar aire puro nadar...,2022.0,1.0
...,...,...,...,...,...,...,...,...,...
3299,club_guatemala,5,hace un anio,Excelente,excelente,[excelente],[excelente],2024.0,1.0
3303,club_guatemala,5,hace 2 anios,bantab,bantab,[bantab],[bantab],2023.0,1.0
3305,club_guatemala,5,hace 5 anios,Desde,desde,[desde],[desde],2020.0,1.0
3306,club_guatemala,5,hace 7 anios,Cicatrización,cicatrizacion,[cicatrizacion],[cicatrizacion],2018.0,1.0


### Guardado

In [205]:
new_df = df[['place_id', 'real_time', 'real_stars', 'sentences', 'lemm_sentences']].reset_index(drop=True)
new_df.rename(columns={'place_id': 'place', 'real_time': 'year', 'real_stars': 'stars'}, inplace=True)

In [206]:
new_df

,place,year,stars,sentences,lemm_sentences
0,club_majadas,2025.0,1.0,"[costo beneficio correcto, limpio y comodo de ...","[costo beneficio correcto, limpio y comodo de ..."
1,club_majadas,2020.0,1.0,"[buena atencion del personal, muy higienico, r...","[buen atencion del personal, mucho higienico, ..."
2,club_majadas,2024.0,1.0,[me encanto el lugar super recomendado la amab...,[yo encantar el lugar super recomendado el ama...
3,club_majadas,2017.0,1.0,[un lugar para ejercitarse en el gimnasio clas...,[uno lugar para ejercitar él en el gimnasio cl...
4,club_majadas,2022.0,1.0,[un bello lugar para respirar aire puro nadar ...,[uno bello lugar para respirar aire puro nadar...
...,...,...,...,...,...
2174,club_guatemala,2024.0,1.0,[excelente],[excelente]
2175,club_guatemala,2023.0,1.0,[bantab],[bantab]
2176,club_guatemala,2020.0,1.0,[desde],[desde]
2177,club_guatemala,2018.0,1.0,[cicatrizacion],[cicatrizacion]


In [207]:
new_df.to_csv('../data/processed/reviews.csv', index=False)